In [14]:
import docker
import time

In [15]:
client = docker.from_env()

In [16]:
drizzle_schema = """
import { integer, pgTable, text } from "drizzle-orm/pg-core";

export const gymEquipmentTable = pgTable("gym_equipment", {
    id: integer().primaryKey().generatedAlwaysAsIdentity(),
    name: text(),
    description: text(),
    price: integer(),
    stock: integer(),
});
""".strip()

In [17]:
def make_drizzle_cmd(schema: str):
    return [
        "sh",
        "-c",
        f"echo '{schema}' > src/db/schema/application.ts && npx drizzle-kit push"
    ]

In [28]:
def test_drizzle(schema: str):
    container = None
    postgres = None
    network = None
    try:
        network = client.networks.create("test-network", driver="bridge")
        postgres = client.containers.run(
            "postgres:17.0-alpine",
            detach=True,
            network="test-network",
            hostname="postgres",
            environment={
                "POSTGRES_USER": "postgres",
                "POSTGRES_PASSWORD": "postgres",
                "POSTGRES_DB": "postgres",
            },
        )
        # run healthchecks until postgres is ready
        while True:
            try:
                is_ready = postgres.exec_run(["pg_isready", "-U", "postgres"])
                if is_ready.exit_code == 0:
                    break
            except docker.errors.APIError:
                time.sleep(0.5)
        container = client.containers.run(
            "botbuild/app_schema",
            make_drizzle_cmd(schema),
            stderr=True,
            detach=True,
            network="test-network",
            environment={"NO_COLOR": "1", "FORCE_COLOR": "0"},
        )
        output = container.attach(stdout=True, stderr=True, stream=True, logs=True)
        stdout = [line.decode("utf-8") for line in output]
        for line in output:
            print(line) 
        result = container.wait()
    finally:
        if container:
            container.remove()
        if postgres:
            postgres.stop()
            postgres.remove()
        if network:
            network.remove()
            
    return (result["StatusCode"], stdout)

In [30]:
code, output = test_drizzle(drizzle_schema)

APIError: 409 Client Error for http+docker://localhost/v1.47/networks/create: Conflict ("network with name test-network already exists")

In [9]:
output

NameError: name 'output' is not defined

In [20]:
with open("templates/tsp_schema/main.tsp", "r") as f:
    tsp_schema = f.read()

In [21]:
def make_typespec_cmd(typespec_definitions: str):
    return [
        "sh",
        "-c",
        f"echo '{typespec_definitions}' > schema.tsp && tsp compile schema.tsp --no-emit"
    ]

In [22]:
container = client.containers.run(
    "botbuild/tsp_compiler",
    make_typespec_cmd(tsp_schema),
    stderr=True,
    detach=True,
    remove=True,
    environment={"NO_COLOR": "1", "FORCE_COLOR": "0"},
)
output = container.attach(stdout=True, stderr=True, stream=True, logs=True)
stdout = [line.decode("utf-8") for line in output]
for line in output:
    print(line) 
result = container.wait()

In [23]:
stdout, result

(['TypeSpec compiler v0.64.0\n\n', 'Compilation completed successfully.\n\n'],
 {'StatusCode': 0})